**Important**: This notebook is different from the other as it directly calls **ImageJ Kappa plugin** using the [`scyjava` ImageJ brige](https://github.com/scijava/scyjava).

Since Kappa uses ImageJ1 features, you might not be able to run this notebook on an headless machine (need to be tested).

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

# Init ImageJ with Fiji plugins
# It can take a while if Java artifacts are not yet cached.
import imagej
java_deps = []
java_deps.append('org.scijava:Kappa:1.7.1')
ij = imagej.init("+".join(java_deps), headless=False)

import jnius

# Load Java classes
KappaFrame = jnius.autoclass('sc.fiji.kappa.gui.KappaFrame')
CurvesExporter = jnius.autoclass('sc.fiji.kappa.gui.CurvesExporter')

# Load ImageJ services
dsio = ij.context.getService(jnius.autoclass('io.scif.services.DatasetIOService'))
dsio = jnius.cast('io.scif.services.DatasetIOService', dsio)

# Set data path
data_dir = Path("/home/hadim/.data/Postdoc/Kappa/sine_curve_SDM")

# Pixel size used when fixed
fixed_pixel_size = 0.16

# Used to select pixels around the initialization curves
base_radius_um = 1.6

enable_control_points_adjustment = True

# "Point Distance Minimization" or "Squared Distance Minimization"
if '_SDM' in data_dir.name:
  fitting_algorithm = "Squared Distance Minimization"
else:
  fitting_algorithm = "Point Distance Minimization"
fitting_algorithm

'Squared Distance Minimization'

In [2]:
experiment_names = ['variable_snr', 'variable_pixel_size', 'variable_initial_position', 'variable_psf_size']
experiment_names = ['variable_psf_size']

for experiment_name in tqdm(experiment_names, total=len(experiment_names)):
    
    experiment_path = data_dir / experiment_name

    fnames = list(experiment_path.glob("*.tif"))
    n = len(fnames)

    for fname in tqdm(fnames, total=n):
        kappa_path = fname.with_suffix(".kapp")
        assert kappa_path.exists(), f'{kappa_path} does not exist.'

        curvatures_path = fname.with_suffix(".csv")
        
        if not curvatures_path.is_file():

          frame = KappaFrame(ij.context)
          frame.getKappaMenubar().openImageFile(str(fname))
          frame.resetCurves()
          frame.getKappaMenubar().loadCurveFile(str(kappa_path))
          frame.getCurves().setAllSelected()

          # Compute threshold according to the image
          dataset = dsio.open(str(fname))
          mean = ij.op().stats().mean(dataset).getRealDouble() 
          std = ij.op().stats().stdDev(dataset).getRealDouble()
          threshold = int(mean + std * 2)

          # Used fixed pixel size or the one in the filename
          if fname.stem.startswith('pixel_size'):
              pixel_size = float(fname.stem.split("_")[-2])
          elif experiment_name == 'variable_psf_size':
              pixel_size = 0.01
          else:
              pixel_size = fixed_pixel_size

          base_radius = int(np.round(base_radius_um / pixel_size))

          # Set curve fitting parameters
          frame.setEnableCtrlPtAdjustment(enable_control_points_adjustment)
          frame.setFittingAlgorithm(fitting_algorithm)
          frame.getInfoPanel().thresholdRadiusSpinner.setValue(ij.py.to_java(base_radius))
          frame.getInfoPanel().thresholdSlider.setValue(threshold)
          frame.getInfoPanel().updateConversionField(str(pixel_size))

          # Fit the curves
          frame.fitCurves()

          # Save fitted curves
          frame.getKappaMenubar().saveCurveFile(str(fname.with_suffix(".FITTED.kapp")))

          # Export results
          exporter = CurvesExporter(frame)
          exporter.exportToFile(str(curvatures_path), False)

          # Remove duplicate rows during CSV export.
          # See https://github.com/brouhardlab/Kappa/issues/12
          df = pd.read_csv(curvatures_path)
          df = df.drop_duplicates()
          df.to_csv(curvatures_path)

In [3]:
5+5

10